In [1]:
# https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-5.%20kogpt2_kornli_tpu.ipynb

# pip install transformers

In [2]:
import transformers
transformers.__version__

'4.35.2'

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import sys

print(os.getcwd())

/content


In [6]:
import pandas as pd
train_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')
dev_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/dev.hate.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/test.hate.no_label.csv')

# , sep='\t', quoting=3

In [7]:
# # 훈련 데이터 다운로드
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

# # 검증 데이터 다운로드
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

# # 테스트 데이터 다운로드
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

In [8]:
# train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
# train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
# val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
# test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [9]:
train_data.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [10]:
LABEL_DIC = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

train_data['label'] = train_data['label'].map(lambda x: LABEL_DIC[x])
dev_data['label'] = dev_data['label'].map(lambda x: LABEL_DIC[x])

In [12]:
# 데이터 정제

import string

## remove basic punctuation
def remove_punc(text):
    text_nopunc = "".join([char for char in text if char not in string.punctuation])
    return text_nopunc

train_data['comments'] = train_data['comments'].apply(lambda x: remove_punc(x))
# train_data['news_title'] = train_data['news_title'].apply(lambda x: remove_punc(x))

dev_data['comments'] = dev_data['comments'].apply(lambda x: remove_punc(x))

test_data['comments'] = test_data['comments'].apply(lambda x: remove_punc(x))

In [13]:
import re

# remove all punctuations except korean, english, and number
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[a-zA-Z]', '', text)
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ]+', '', text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    return text

train_data['comments'] = train_data['comments'].apply(cleanse)
# train['news_title'] = train['news_title'].apply(cleanse)

dev_data['comments'] = dev_data['comments'].apply(cleanse)

test_data['comments'] = test_data['comments'].apply(cleanse)

train_data.head()

,comments,label
0,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,2
1,한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요,0
2,못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지그래야 공정한 사회지심은...,2
3,12화 어설펐는데 34화 지나서부터는 갈수록 너무 재밌던데,0
4,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,2


In [15]:
train_data_list = []

for q, label in zip(train_data['comments'], train_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train_data_list.append(data)

In [16]:
dev_data_list = []

for q, label in zip(dev_data['comments'], dev_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dev_data_list.append(data)

In [17]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')

In [18]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031]


In [19]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카', '리스', '마', '▁없는', '▁악', '역']


In [20]:
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'보는내내 그대로 들어맞는 예측 카리스마 없는 악역'

In [21]:
for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

보는
내
내
그대로
들어
맞
는
예측
카
리스
마
없는
악
역


In [22]:
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

['▁전', '율을', '▁일으키는', '▁영화', '.', '▁다시', '▁보고', '싶', '은', '▁영화']


In [23]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584]


In [24]:
for elem in tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"):
  print(tokenizer.decode(elem))

전
율을
일으키는
영화
.
다시
보고
싶
은
영화


In [25]:
for elem in tokenizer.encode("happy birthday~!"):
  print(tokenizer.decode(elem))


ha
pp
y
b
ir
th
d
ay
~
!


In [26]:
print(tokenizer.decode(0))
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

<s>
</s>
<usr>
<pad>
<sys>


In [27]:
max_seq_len = 140

In [28]:
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, padding='max_length')
print(encoded_result)
print('길이 :', len(encoded_result))

[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 140


In [29]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=128))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584]


In [30]:
def convert_examples_to_features(comments, max_seq_len, tokenizer):

    input_ids = []

    for sent1 in tqdm(comments, total = len(comments)):
        # bos_token = [tokenizer.bos_token]
        # eos_token = [tokenizer.eos_token]
        # sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token + ['']

        sent1_tokens = [tokenizer.bos_token] + tokenizer.tokenize(sent1) + [tokenizer.eos_token]

        tokens = sent1_tokens
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids

In [31]:
X_train = convert_examples_to_features(train_data['comments'], max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 7896/7896 [00:06<00:00, 1213.17it/s]


In [32]:
# 최대 길이: 128
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :', input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    1  9490 18146 17653  9327  8194  9050 11050  9769 23426 22296  8022
  9673  7641  7373  7498  6889 18146  7497  9124  7777  6883  9668 31040
  9716 16882  7600  7098     1     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 140
정수 인코딩 복원 : </s> 현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [33]:
X_dev = convert_examples_to_features(dev_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 471/471 [00:00<00:00, 3600.26it/s]


In [34]:
# 최대 길이: 128
input_id = X_dev[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    1  9697  8243  6958 10343 29960 18882  7655  7182  9877  8765 16035
  9038  9677  7978  7182     1     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 140
정수 인코딩 복원 : </s> 송중기 시대극은 믿고본다 첫회 신선하고 좋았다</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [35]:
X_test = convert_examples_to_features(test_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 974/974 [00:00<00:00, 4137.69it/s]


In [36]:
train_label = train_data['label'].tolist()
dev_label = dev_data['label'].tolist()
# test_label = test_data['label'].tolist()

In [37]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_dev = idx_encode.transform(dev_label) # 고유한 정수로 변환
# y_test = idx_encode.transform(test_label) # 고유한 정수로 변환

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{0: 0, 1: 1, 2: 2}
{0: 0, 1: 1, 2: 2}


In [38]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [39]:
max_seq_len = 140

In [40]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [41]:
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 140, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [42]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 140, 768), dtype=tf.float32, name=None), name='tfgpt2_model/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model'")


In [43]:
print(outputs[1])

(<KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by 

In [44]:
print(outputs[0][:, -1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")


In [45]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

TPU 사용법 : https://wikidocs.net/119990

In [46]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [47]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [48]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [49]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta = 0.001,
    patience = 2)

model.fit(
    X_train, y_train, epochs=10, batch_size=64, validation_data=(X_dev, y_dev),
    callbacks = [early_stopping]
)

Epoch 1/10
124/124 [==============================] - 84s 219ms/step - loss: 1.3289 - accuracy: 0.4111 - val_loss: 1.0219 - val_accuracy: 0.4692
Epoch 2/10
124/124 [==============================] - 16s 128ms/step - loss: 0.9187 - accuracy: 0.5503 - val_loss: 0.9626 - val_accuracy: 0.5223
Epoch 3/10
124/124 [==============================] - 16s 129ms/step - loss: 0.7443 - accuracy: 0.6550 - val_loss: 0.9154 - val_accuracy: 0.5732
Epoch 4/10
124/124 [==============================] - 16s 129ms/step - loss: 0.5946 - accuracy: 0.7408 - val_loss: 0.8329 - val_accuracy: 0.6263
Epoch 5/10
124/124 [==============================] - 16s 128ms/step - loss: 0.4197 - accuracy: 0.8241 - val_loss: 1.1449 - val_accuracy: 0.5881
Epoch 6/10
124/124 [==============================] - 21s 169ms/step - loss: 0.3010 - accuracy: 0.8822 - val_loss: 1.2903 - val_accuracy: 0.5881


In [50]:
# 토큰화 된 상태
# X_test

In [51]:
output = model.predict(X_test)

31/31 [==============================] - 12s 48ms/step


In [52]:
# output

In [53]:
import torch

outputTensor = torch.Tensor(output)

In [54]:
# outputTensor = nn.Softmax(dim=1)(outputTensor)
pred = torch.argmax(outputTensor, dim=-1)

In [55]:
pred

tensor([2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 2, 1, 0, 2, 1, 1, 1, 1, 2, 0, 1, 1, 0, 0,
        1, 0, 2, 0, 1, 0, 2, 0, 1, 1, 1, 0, 0, 2, 2, 1, 1, 2, 1, 1, 0, 2, 1, 1,
        1, 2, 2, 2, 1, 1, 0, 2, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 2, 1, 2, 0, 0, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 2, 2, 0, 0, 1, 0, 0, 2, 1, 1, 1, 1, 1, 0, 0, 1,
        2, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
        2, 1, 2, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 2, 0,
        0, 2, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 2, 0, 2, 1, 0, 0, 0,
        0, 1, 2, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 0, 0, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 2, 1, 1, 0, 1, 0, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
        1, 1, 2, 1, 1, 2, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 1,
        0, 1, 0, 1, 1, 0, 2, 1, 0, 2, 0, 1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 1, 2, 0,
        1, 2, 1, 2, 1, 0, 0, 0, 1, 0, 1,

In [56]:
test_label = [int(k.cpu()) for k in pred]
test_data['label'] = test_label

In [57]:
test_data.to_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/submission_new_kogpt2_cleaning.csv', index=None)